<a href="https://colab.research.google.com/github/sowrab9949/Major-Project-Sentiment-Analysis/blob/master/Major_Project_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install nltk

In [21]:
from sklearn.datasets import load_files
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [5]:
df = pd.read_table('/content/drive/My Drive/smartknower/Machine learning/Sentiment Analysis project/Restaurant_Reviews.tsv')
X, y = df.Review, df.	Liked

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  1000 non-null   object
 1   Liked   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [7]:
df['Review'].value_counts()

I won't be back.                                                                    2
I love this place.                                                                  2
I would not recommend this place.                                                   2
The food was terrible.                                                              2
The chefs were friendly and did a good job.                                         1
                                                                                   ..
Not much flavor to them, and very poorly constructed.                               1
We've tried to like this place but after 10+ times I think we're done with them.    1
I waited and waited.                                                                1
Definitely not worth the $3 I paid.                                                 1
The block was amazing.                                                              1
Name: Review, Length: 996, dtype: int64

In [8]:
df['Review'][5]

'Now I am getting angry and I want my damn pho.'

In [9]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [10]:
nltk.download('stopwords')
corpus = []
for i in range(0, 1000):
    comment = re.sub('[^a-zA-Z]', ' ', df['Review'][i])
    comment = comment.lower()
    comment = comment.split()
    ps = PorterStemmer()
    comment = [ps.stem(word) for word in comment if not word in set(stopwords.words('english'))]
    comment = ' '.join(comment)
    corpus.append(comment)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:
tfidf = TfidfVectorizer(max_features=1500)
x = tfidf.fit_transform(corpus).toarray()
y = df.iloc[:, 1].values

In [23]:
pickle.dump(tfidf, open('tfidf-transform.pkl', 'wb'))

In [24]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state= 0)

In [25]:
text_model = MultinomialNB(alpha=0.2)
text_model.fit(x_train,y_train)

MultinomialNB(alpha=0.2, class_prior=None, fit_prior=True)

In [26]:
filename = 'restaurant-sentiment-mnb-model.pkl'
pickle.dump(text_model, open(filename, 'wb'))

In [27]:
y_pred = text_model.predict(x_test)

In [ ]:
y_test

In [16]:
accuracy_score(y_test,y_pred)

0.752

In [17]:
confusion_matrix(y_test,y_pred)

array([[ 86,  31],
       [ 31, 102]])

In [18]:
pd.crosstab(y_test,y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
Actual,,,
0,86,31,117
1,31,102,133
All,117,133,250


In [19]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.74      0.74      0.74       117
           1       0.77      0.77      0.77       133

    accuracy                           0.75       250
   macro avg       0.75      0.75      0.75       250
weighted avg       0.75      0.75      0.75       250

